In [1]:
import pandas as pd
import numpy as np
import torch
import json
import os
from Network.deepnetwork2 import *
from Memory.dataset import *
from torch.utils.data import DataLoader
from Utils.utils import *
from sklearn.metrics import precision_score, recall_score
import torch.nn as nn

In [15]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('TensorBoard/1stImitationAgent_Action0_OldData_LSTM')


In [3]:
# dataset = Feeder6('Log/Train/',range(56))

100%|██████████| 69822/69822 [05:25<00:00, 214.59it/s]


In [11]:
dataset = torch.load('Memory/Datasets/Feeder_Action0')
# torch.save(dataset,'Memory/Datasets/OldData/Feeder_ActionAttackMinion_0_NewData')

In [12]:
dataloader = DataLoader(dataset,batch_size = 256, num_workers = 20,shuffle=True)

In [6]:
def accuracy(pred, label):
    pred = torch.argmax(pred, dim=1).long()
#     print(pred)
    label = torch.argmax(label,dim=1).long()
    acc = torch.mean((pred == label).float())
    pred = pred.detach().cpu().numpy()
    label = label.detach().cpu().numpy()
    p = precision_score(label, pred,average='micro')
    r = recall_score(label, pred,average='micro')
    return p,r,acc 

class AverageMeter(object):

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [13]:
for i, (data,reward,label) in enumerate(dataloader):
    print(reward.shape)
    print(data.shape)
    break

torch.Size([256, 55])
torch.Size([256, 516])


In [14]:
model = ImitationNetwork_Fuzzy(0.1,55,'Test',516,'Test')
model.to(torch.device('cuda:1'))

ImitationNetwork_Fuzzy(
  (dense1): Linear(in_features=516, out_features=1024, bias=True)
  (dense3): Linear(in_features=1024, out_features=512, bias=True)
  (dense5): Linear(in_features=512, out_features=128, bias=True)
  (dense6): Linear(in_features=128, out_features=64, bias=True)
  (dense8): Linear(in_features=64, out_features=55, bias=True)
  (loss): CrossEntropyLoss()
)

In [16]:
crit = nn.BCEWithLogitsLoss().to(torch.device('cuda:1'))
losses = 0.0
accs  = 0.0
precisions  = AverageMeter()
recalls  = AverageMeter()

opt = torch.optim.SGD(model.parameters(), 0.01, 
                          momentum=0.9, 
                          weight_decay=1e-4) 
# checkpoint = torch.load('Models/1stImitationAgent_OldData.ckpt')
# model.load_state_dict(checkpoint)
for epoch in range(20):
    for i, (data,reward,label) in enumerate(dataloader):
        data_,reward_,label_ = map(lambda x:x.to(torch.device('cuda:1')),(data,reward,label))

        pred = model.forward(data_.float())
#         y_onehot = label.detach().numpy()
#         y_onehot = (np.arange(56) == y_onehot[:,None]).astype(np.float32)
#         y_onehot = torch.from_numpy(y_onehot).to(torch.device('cuda:1'))
        loss = crit(pred,torch.nn.functional.softmax(reward_))
        p,r, acc = accuracy(pred, torch.nn.functional.softmax(reward_))

        opt.zero_grad()
        loss.backward()
        opt.step()

#         losses.update(loss.item(),530)
#         accs.update(acc.item(),530)
        losses += loss.item()
        accs += acc.item()
        precisions.update(p, 530)
        recalls.update(r,530)

        if i % 5 == 0:
            print('Epoch:[{0}][{1}/{2}]\t'
#                   'Loss {losses.val:.3f} ({losses.avg:.3f})\t'
#                   'Accuracy {accs.val:.3f} ({accs.avg:.3f})\t'
                  'Loss {losses:.3f} \t'
                  'Accuracy {accs:.3f}\t'
                  'Precison {precisions.val:.3f} ({precisions.avg:.3f})\t'
                  'Recall {recalls.val:.3f} ({recalls.avg:.3f})'.format(
                        epoch,i,len(dataloader) ,losses=loss, accs=acc, 
                        precisions=precisions, recalls=recalls))
        if i % 10 == 0:
            torch.save(model.state_dict(), 'Models/1stImitationAgent_OldData_Action0.ckpt')
#             # ...log the running loss
            writer.add_scalar('training loss',
                            losses / 10,
                            epoch * len(dataloader) + i)

            # ...log a Matplotlib Figure showing the model's predictions on a
            # random mini-batch
            writer.add_scalar('accuracy',
                            accs / 10,
                            global_step=epoch * len(dataloader) + i)
            losses = 0.0
            accs = 0.0
    #         print('Epoch:[{0}][{1}/{2}]\t'
    #               'Loss {losses:.3f} \t'
    #               'Accuracy {accs:.3f}\t'
    #               'Precison {precisions:.3f}\t'
    #               'Recall {recalls:.3f}'.format(
    #                     epoch, losses=loss, accs=acc, 
    #                     precisions=p, recalls=r))
    

/home/kietcdx/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/kietcdx/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch:[0][0/195]	Loss 0.694 	Accuracy 0.012	Precison 0.012 (0.012)	Recall 0.012 (0.012)
Epoch:[0][5/195]	Loss 0.685 	Accuracy 0.023	Precison 0.023 (0.010)	Recall 0.023 (0.010)
Epoch:[0][10/195]	Loss 0.667 	Accuracy 0.000	Precison 0.000 (0.007)	Recall 0.000 (0.007)
Epoch:[0][15/195]	Loss 0.636 	Accuracy 0.000	Precison 0.000 (0.005)	Recall 0.000 (0.005)
Epoch:[0][20/195]	Loss 0.579 	Accuracy 0.004	Precison 0.004 (0.004)	Recall 0.004 (0.004)
Epoch:[0][25/195]	Loss 0.471 	Accuracy 0.000	Precison 0.000 (0.003)	Recall 0.000 (0.003)
Epoch:[0][30/195]	Loss 0.283 	Accuracy 0.000	Precison 0.000 (0.003)	Recall 0.000 (0.003)
Epoch:[0][35/195]	Loss 0.123 	Accuracy 0.000	Precison 0.000 (0.003)	Recall 0.000 (0.003)
Epoch:[0][40/195]	Loss 0.109 	Accuracy 0.023	Precison 0.023 (0.004)	Recall 0.023 (0.004)
Epoch:[0][45/195]	Loss 0.121 	Accuracy 0.172	Precison 0.172 (0.016)	Recall 0.172 (0.016)
Epoch:[0][50/195]	Loss 0.116 	Accuracy 0.215	Precison 0.215 (0.033)	Recall 0.215 (0.033)
Epoch:[0][55/195]	Loss 

Epoch:[2][80/195]	Loss 0.088 	Accuracy 0.156	Precison 0.156 (0.169)	Recall 0.156 (0.169)
Epoch:[2][85/195]	Loss 0.089 	Accuracy 0.156	Precison 0.156 (0.169)	Recall 0.156 (0.169)
Epoch:[2][90/195]	Loss 0.089 	Accuracy 0.145	Precison 0.145 (0.169)	Recall 0.145 (0.169)
Epoch:[2][95/195]	Loss 0.089 	Accuracy 0.168	Precison 0.168 (0.169)	Recall 0.168 (0.169)
Epoch:[2][100/195]	Loss 0.089 	Accuracy 0.207	Precison 0.207 (0.169)	Recall 0.207 (0.169)
Epoch:[2][105/195]	Loss 0.088 	Accuracy 0.203	Precison 0.203 (0.170)	Recall 0.203 (0.170)
Epoch:[2][110/195]	Loss 0.089 	Accuracy 0.184	Precison 0.184 (0.170)	Recall 0.184 (0.170)
Epoch:[2][115/195]	Loss 0.088 	Accuracy 0.207	Precison 0.207 (0.170)	Recall 0.207 (0.170)
Epoch:[2][120/195]	Loss 0.089 	Accuracy 0.215	Precison 0.215 (0.170)	Recall 0.215 (0.170)
Epoch:[2][125/195]	Loss 0.088 	Accuracy 0.195	Precison 0.195 (0.170)	Recall 0.195 (0.170)
Epoch:[2][130/195]	Loss 0.088 	Accuracy 0.168	Precison 0.168 (0.170)	Recall 0.168 (0.170)
Epoch:[2][135/

Epoch:[4][155/195]	Loss 0.088 	Accuracy 0.148	Precison 0.148 (0.176)	Recall 0.148 (0.176)
Epoch:[4][160/195]	Loss 0.088 	Accuracy 0.207	Precison 0.207 (0.176)	Recall 0.207 (0.176)
Epoch:[4][165/195]	Loss 0.089 	Accuracy 0.141	Precison 0.141 (0.176)	Recall 0.141 (0.176)
Epoch:[4][170/195]	Loss 0.085 	Accuracy 0.219	Precison 0.219 (0.176)	Recall 0.219 (0.176)
Epoch:[4][175/195]	Loss 0.087 	Accuracy 0.230	Precison 0.230 (0.176)	Recall 0.230 (0.176)
Epoch:[4][180/195]	Loss 0.087 	Accuracy 0.211	Precison 0.211 (0.176)	Recall 0.211 (0.176)
Epoch:[4][185/195]	Loss 0.087 	Accuracy 0.207	Precison 0.207 (0.177)	Recall 0.207 (0.177)
Epoch:[4][190/195]	Loss 0.087 	Accuracy 0.188	Precison 0.188 (0.177)	Recall 0.188 (0.177)
Epoch:[5][0/195]	Loss 0.088 	Accuracy 0.215	Precison 0.215 (0.177)	Recall 0.215 (0.177)
Epoch:[5][5/195]	Loss 0.088 	Accuracy 0.203	Precison 0.203 (0.177)	Recall 0.203 (0.177)
Epoch:[5][10/195]	Loss 0.087 	Accuracy 0.188	Precison 0.188 (0.177)	Recall 0.188 (0.177)
Epoch:[5][15/19

Epoch:[7][30/195]	Loss 0.085 	Accuracy 0.406	Precison 0.406 (0.214)	Recall 0.406 (0.214)
Epoch:[7][35/195]	Loss 0.083 	Accuracy 0.430	Precison 0.430 (0.215)	Recall 0.430 (0.215)
Epoch:[7][40/195]	Loss 0.086 	Accuracy 0.410	Precison 0.410 (0.215)	Recall 0.410 (0.215)
Epoch:[7][45/195]	Loss 0.087 	Accuracy 0.414	Precison 0.414 (0.216)	Recall 0.414 (0.216)
Epoch:[7][50/195]	Loss 0.086 	Accuracy 0.406	Precison 0.406 (0.217)	Recall 0.406 (0.217)
Epoch:[7][55/195]	Loss 0.086 	Accuracy 0.383	Precison 0.383 (0.217)	Recall 0.383 (0.217)
Epoch:[7][60/195]	Loss 0.086 	Accuracy 0.426	Precison 0.426 (0.218)	Recall 0.426 (0.218)
Epoch:[7][65/195]	Loss 0.086 	Accuracy 0.387	Precison 0.387 (0.218)	Recall 0.387 (0.218)
Epoch:[7][70/195]	Loss 0.085 	Accuracy 0.434	Precison 0.434 (0.219)	Recall 0.434 (0.219)
Epoch:[7][75/195]	Loss 0.087 	Accuracy 0.375	Precison 0.375 (0.220)	Recall 0.375 (0.220)
Epoch:[7][80/195]	Loss 0.086 	Accuracy 0.379	Precison 0.379 (0.220)	Recall 0.379 (0.220)
Epoch:[7][85/195]	Los

Epoch:[9][100/195]	Loss 0.083 	Accuracy 0.414	Precison 0.414 (0.262)	Recall 0.414 (0.262)
Epoch:[9][105/195]	Loss 0.085 	Accuracy 0.488	Precison 0.488 (0.262)	Recall 0.488 (0.262)
Epoch:[9][110/195]	Loss 0.084 	Accuracy 0.438	Precison 0.438 (0.263)	Recall 0.438 (0.263)
Epoch:[9][115/195]	Loss 0.084 	Accuracy 0.402	Precison 0.402 (0.263)	Recall 0.402 (0.263)
Epoch:[9][120/195]	Loss 0.084 	Accuracy 0.477	Precison 0.477 (0.263)	Recall 0.477 (0.263)
Epoch:[9][125/195]	Loss 0.084 	Accuracy 0.469	Precison 0.469 (0.264)	Recall 0.469 (0.264)
Epoch:[9][130/195]	Loss 0.085 	Accuracy 0.402	Precison 0.402 (0.264)	Recall 0.402 (0.264)
Epoch:[9][135/195]	Loss 0.084 	Accuracy 0.461	Precison 0.461 (0.265)	Recall 0.461 (0.265)
Epoch:[9][140/195]	Loss 0.085 	Accuracy 0.391	Precison 0.391 (0.265)	Recall 0.391 (0.265)
Epoch:[9][145/195]	Loss 0.084 	Accuracy 0.375	Precison 0.375 (0.265)	Recall 0.375 (0.265)
Epoch:[9][150/195]	Loss 0.083 	Accuracy 0.367	Precison 0.367 (0.266)	Recall 0.367 (0.266)
Epoch:[9][

Epoch:[11][165/195]	Loss 0.084 	Accuracy 0.395	Precison 0.395 (0.292)	Recall 0.395 (0.292)
Epoch:[11][170/195]	Loss 0.084 	Accuracy 0.414	Precison 0.414 (0.293)	Recall 0.414 (0.293)
Epoch:[11][175/195]	Loss 0.084 	Accuracy 0.445	Precison 0.445 (0.293)	Recall 0.445 (0.293)
Epoch:[11][180/195]	Loss 0.085 	Accuracy 0.449	Precison 0.449 (0.293)	Recall 0.449 (0.293)
Epoch:[11][185/195]	Loss 0.082 	Accuracy 0.418	Precison 0.418 (0.293)	Recall 0.418 (0.293)
Epoch:[11][190/195]	Loss 0.083 	Accuracy 0.457	Precison 0.457 (0.294)	Recall 0.457 (0.294)
Epoch:[12][0/195]	Loss 0.084 	Accuracy 0.402	Precison 0.402 (0.294)	Recall 0.402 (0.294)
Epoch:[12][5/195]	Loss 0.083 	Accuracy 0.422	Precison 0.422 (0.294)	Recall 0.422 (0.294)
Epoch:[12][10/195]	Loss 0.083 	Accuracy 0.406	Precison 0.406 (0.294)	Recall 0.406 (0.294)
Epoch:[12][15/195]	Loss 0.086 	Accuracy 0.441	Precison 0.441 (0.295)	Recall 0.441 (0.295)
Epoch:[12][20/195]	Loss 0.084 	Accuracy 0.414	Precison 0.414 (0.295)	Recall 0.414 (0.295)
Epoch:

Epoch:[14][35/195]	Loss 0.085 	Accuracy 0.379	Precison 0.379 (0.313)	Recall 0.379 (0.313)
Epoch:[14][40/195]	Loss 0.083 	Accuracy 0.441	Precison 0.441 (0.314)	Recall 0.441 (0.314)
Epoch:[14][45/195]	Loss 0.085 	Accuracy 0.379	Precison 0.379 (0.314)	Recall 0.379 (0.314)
Epoch:[14][50/195]	Loss 0.082 	Accuracy 0.418	Precison 0.418 (0.314)	Recall 0.418 (0.314)
Epoch:[14][55/195]	Loss 0.086 	Accuracy 0.414	Precison 0.414 (0.314)	Recall 0.414 (0.314)
Epoch:[14][60/195]	Loss 0.084 	Accuracy 0.363	Precison 0.363 (0.314)	Recall 0.363 (0.314)
Epoch:[14][65/195]	Loss 0.083 	Accuracy 0.391	Precison 0.391 (0.314)	Recall 0.391 (0.314)
Epoch:[14][70/195]	Loss 0.083 	Accuracy 0.492	Precison 0.492 (0.315)	Recall 0.492 (0.315)
Epoch:[14][75/195]	Loss 0.084 	Accuracy 0.453	Precison 0.453 (0.315)	Recall 0.453 (0.315)
Epoch:[14][80/195]	Loss 0.085 	Accuracy 0.422	Precison 0.422 (0.315)	Recall 0.422 (0.315)
Epoch:[14][85/195]	Loss 0.084 	Accuracy 0.395	Precison 0.395 (0.315)	Recall 0.395 (0.315)
Epoch:[14]

Epoch:[16][100/195]	Loss 0.083 	Accuracy 0.359	Precison 0.359 (0.329)	Recall 0.359 (0.329)
Epoch:[16][105/195]	Loss 0.084 	Accuracy 0.414	Precison 0.414 (0.329)	Recall 0.414 (0.329)
Epoch:[16][110/195]	Loss 0.083 	Accuracy 0.406	Precison 0.406 (0.329)	Recall 0.406 (0.329)
Epoch:[16][115/195]	Loss 0.083 	Accuracy 0.430	Precison 0.430 (0.329)	Recall 0.430 (0.329)
Epoch:[16][120/195]	Loss 0.084 	Accuracy 0.422	Precison 0.422 (0.329)	Recall 0.422 (0.329)
Epoch:[16][125/195]	Loss 0.081 	Accuracy 0.430	Precison 0.430 (0.329)	Recall 0.430 (0.329)
Epoch:[16][130/195]	Loss 0.083 	Accuracy 0.418	Precison 0.418 (0.330)	Recall 0.418 (0.330)
Epoch:[16][135/195]	Loss 0.082 	Accuracy 0.469	Precison 0.469 (0.330)	Recall 0.469 (0.330)
Epoch:[16][140/195]	Loss 0.083 	Accuracy 0.391	Precison 0.391 (0.330)	Recall 0.391 (0.330)
Epoch:[16][145/195]	Loss 0.085 	Accuracy 0.430	Precison 0.430 (0.330)	Recall 0.430 (0.330)
Epoch:[16][150/195]	Loss 0.083 	Accuracy 0.406	Precison 0.406 (0.330)	Recall 0.406 (0.330)

Epoch:[18][165/195]	Loss 0.083 	Accuracy 0.441	Precison 0.441 (0.340)	Recall 0.441 (0.340)
Epoch:[18][170/195]	Loss 0.084 	Accuracy 0.422	Precison 0.422 (0.340)	Recall 0.422 (0.340)
Epoch:[18][175/195]	Loss 0.081 	Accuracy 0.492	Precison 0.492 (0.340)	Recall 0.492 (0.340)
Epoch:[18][180/195]	Loss 0.083 	Accuracy 0.465	Precison 0.465 (0.340)	Recall 0.465 (0.340)
Epoch:[18][185/195]	Loss 0.085 	Accuracy 0.398	Precison 0.398 (0.340)	Recall 0.398 (0.340)
Epoch:[18][190/195]	Loss 0.081 	Accuracy 0.418	Precison 0.418 (0.341)	Recall 0.418 (0.341)
Epoch:[19][0/195]	Loss 0.083 	Accuracy 0.438	Precison 0.438 (0.341)	Recall 0.438 (0.341)
Epoch:[19][5/195]	Loss 0.081 	Accuracy 0.426	Precison 0.426 (0.341)	Recall 0.426 (0.341)
Epoch:[19][10/195]	Loss 0.083 	Accuracy 0.387	Precison 0.387 (0.341)	Recall 0.387 (0.341)
Epoch:[19][15/195]	Loss 0.083 	Accuracy 0.379	Precison 0.379 (0.341)	Recall 0.379 (0.341)
Epoch:[19][20/195]	Loss 0.081 	Accuracy 0.348	Precison 0.348 (0.341)	Recall 0.348 (0.341)
Epoch: